# **Introduktion**

Att vara anonym på internet gör det möjligt för människor att säga väldigt hatfulla saker som de normalt inte hade sagt i person. Dessa hatfulla kommentarer kan ha en stor effekt på människor och därför är syftet med detta projekt att försöka filtrera ut hatfulla kommentarer. Detta projekt är baserat på en öppen tävling på Kaggle vid namn "Jigsaw Multilingual Toxic Comment Classification" - alltså så skall en klassifiering göras av hatfulla kommentarer i flertal språk. Genom att identifiera och filtrera hatfulla kommentarer online så kan vi få en säkrare online-upplevelse som leder till högre produktivitet och nöje.

Projektet utförs här på Kaggle för enkelhetens skull då vid försök att använda collab för att lösa problemet så har flertal problem uppstått.

Kaggle Tävling Länk:

https://www.kaggle.com/c/jigsaw-multilingual-toxic-comment-classification/data

# Metod

Vi börjar som vanligt med att hämta nödvändiga bibliotek

In [ ]:
%matplotlib inline

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import GRU,SimpleRNN,LSTM
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping

import sys
import os
import numpy as np
import pandas as pd
import IPython
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from wordcloud import WordCloud

**Data undersökning och visualisering**

Det är viktigt att vi undersöker vår data så att vi enklare kan arbeta med den och nå en så bra lösning som möjligt. Enligt själva tävlingens instruktioner så är den primära datan för tävlingen, i varje tillgänglig fil, själva comment_text kolumnen. Denna kolumn innehåller texten av en kommentar som har blivit klassificerad som 'toxic' eller 'non-toxic' (0 eller 1 i toxic kolumnen). Vårt träning set's kommentarer är enbart på engelska och kommer antingen från 'Civil' kommentarer eller Wikipedia talk page edits. Vårt test set's kommentarer är dock inte i engelska och består istället av ett flertal andra språk.

**Hämtar tränings, validerings och test dataset (CSV filer) **

In [ ]:
valid = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation-processed-seqlen128.csv")
train = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train-processed-seqlen128.csv")
test = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test-processed-seqlen128.csv")
submit = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv")
train = train[['id', 'comment_text', 'input_word_ids', 'input_mask','all_segment_id', 'toxic']].iloc[:20000] #limit på tränings set

Vi kikar på vårt tränings dataset och dess format.

In [ ]:
train.info()

Här ser vi tabellen för vårt träningsdata och därmed några exempel på hur kommentarerna kan se ut. Vi ser att alla är, som tidigare nämnt, på engelska och att vissa är hatfulla (toxic) och vissa inte.

In [ ]:
train.tail(12)

När vi tittar på vårt valideringsdata så ser vi att kommentarerna nu istället är i språken turkiska, spanska och italienska. Precis som tidigare så är kommentarerna även här antingen hatfulla eller inte. Dessa tre språk är inte de ända i denna data, enligt tävlingens sida så ser uppdelningen ut såhär för vårt valideringsdata:

Turkiska = 38%

Spanska = 31%

Andra språk = 31%

In [ ]:
valid.tail(12)

Vårt testdata visar här kommentarer med flera olika språk.

In [ ]:
test.tail(12)

Det kan vara bra om vi ser om det finns null värden i vår testdata. Om det finns null värden så bör vi ersätta dom innan vi går vidare - om vi lämnar null värdena som dom är så kan det skapa problem för oss i senare stadier.

In [ ]:
train.isnull().any(),test.isnull().any() # test om det finns null värden i vår träningsdata

Det verkar som att vi inte behöver ta itu med några null värden, vilket är positivt.

I ovanstående tabeller så har vi sett att det finns både hatfulla och icke-hatfulla kommentarer i vår tränings- och validerings dataset, dock så verkar det som att de flesta kommentarer i vår data är icke-hatfulla. Nedanstående kod är till för att belysa hur uppdelningen faktiskt ser ut rent procentmässigt.

Vi ser att vårt tränings set består av cirka 85-90% icke-hatfulla kommentarer och därmed ca. 10-15% kommentarer som är hatfulla. Vår validerings set är närmare 80-82% när det kommer till icke-hatfulla kommentarer och ca. 18% hatfulla kommentarer.

In [ ]:
train_distribution = train["toxic"].value_counts().values
valid_distribution = valid["toxic"].value_counts().values

non_toxic = [train_distribution[0] / sum(train_distribution) * 100, valid_distribution[0] / sum(valid_distribution) * 100]
toxic = [train_distribution[1] / sum(train_distribution) * 100, valid_distribution[1] / sum(valid_distribution) * 100]

plt.figure(figsize=(9,6))
plt.bar([0, 1], non_toxic, alpha=.4, color="r", width=0.35, label="non-toxic")
plt.bar([0.4, 1.4], toxic, alpha=.4, width=0.35, label="toxic")
plt.xlabel("Dataset")
plt.ylabel("Percentage")
plt.xticks([0.2, 1.2], ["train", "valid"])
plt.legend(loc="upper right")

plt.show()

Istället för att gå på ögonmått så kan vi ta fram de exakta procentvärdena för vår data...

Vi vet nu att vår träningsdata består av 9,74% hatfulla kommentarer och att i vår valideringsdata så är detta värde istället 15,38%.

In [ ]:
print(f"Träningsdata: \nnon-toxic rate: {train_distribution[0] / sum(train_distribution) * 100: .2f} %\ntoxic rate: {train_distribution[1] / sum(train_distribution) * 100: .2f} %")
print(f"Valideringsdata: \nnon-toxic rate: {valid_distribution[0] / sum(valid_distribution) * 100: .2f} %\ntoxic rate: {valid_distribution[1] / sum(valid_distribution) * 100: .2f} %")

Då kommentarerna är texter i olika storlekar så är jag nyfiken om hur mycket längden på kommentarerna faktiskt varierar i vår data.

In [ ]:
# möjliggör unndersökning av längden på kommentarerna i träningsdata
train['char_length'] = train['comment_text'].apply(lambda x: len(str(x)))

Tittar vi på vårt histogram ser vi att de flesta kommentarerna är inom 500 tecken samtidigt som vissa går mot 2000+ tecken.

In [ ]:
# visar plottad histogram för längd på kommentarer (antal tecken)
sns.set()
train['char_length'].hist()
plt.show()

Wordcloud är en funktion som gör det möjligt att se vilka ord som är mest frekventa i en datamängd av texter, vilket är perfekt att använda i vår sits där vi undersöker hatfulla ord bland en större mängd icke-hatfulla ord.

Tittar vi vår Wordcloud nedan ser vi att de mest frekventa orden (större) är icke-hatfulla med ord som *article, page, wikipedia* och att det är svårare att hitta hatfulla ord bland dessa. Detta överensstämmer med vår tidigare framtagna statistik om uppdelningen av hatfulla och icke-hatfulla kommentarer i vår träningsdata.

In [ ]:
def nonan(x):
    if type(x) == str:
        return x.replace("\n", "")
    else:
        return ""

text = ' '.join([nonan(abstract) for abstract in train["comment_text"]])
wordcloud = WordCloud(max_font_size=None, background_color='black', collocations=False,
                      width=1200, height=1000).generate(text)
fig = px.imshow(wordcloud)
fig.update_layout(title_text='Frekventa ord i kommentarer')

Då det är svårt att hitta hatfulla kommentarer bland de icke-hatfulla i en Wordcloud som inte separerar bland de två så gör vi nu två stycken nya Wordcloud's.

Våra första Wordcloud visar nu endast ord som är frekventa i icke-hatfulla kommentarer, dessa ord känner vi igen från förra Wordclouden.

Våran andra Worldcloud visar därmed endast frekventa ord i hatfulla kommentarer.

Det blir tydligt att kommentarer som har klassats som icke-hatfulla inte innehåller någon form av svärord eller liknande medans kommentarer som har klassats som hatfulla innehåller en mängd olika ord som kan klassas som hatfulla. T.ex. 'fuck', 'ni**er', 'dickhead' och 'cunt'.

In [ ]:
subset = train.query("toxic == 0")
text = subset.comment_text.values
wc = WordCloud(background_color="black",max_words=1500)
wc.generate(" ".join(text))
plt.figure(figsize=(7.5, 7.5))
plt.axis("off")
plt.title("Frekventa ord i icke-hatfulla kommentarer", fontsize=16)
plt.imshow(wc.recolor(colormap= 'viridis' , random_state=17))
plt.show()

subset = train.query("toxic == 1")
text = subset.comment_text.values
wc = WordCloud(background_color="black",max_words=1500)
wc.generate(" ".join(text))
plt.figure(figsize=(7.5, 7.5))
plt.axis("off")
plt.title("Frekventa ord i hatfulla kommentarer", fontsize=16)
plt.imshow(wc.recolor(colormap= 'viridis' , random_state=17))
plt.show()

**Modellering**

Vårt tränings set delar upp hatfulla kommentarer i olika klasser:

severe toxic

obscene

threat

insult

identity hate

För att göra det lättare att tackla detta problem så kommer jag att ta bort dessa 5 klassificeringar och istället tackla problemet som ett binärt klassificerings problem - d.v.s. så är antingen en kommentar hatfull eller inte. Samt så kommer vi att göra vår träning på en delmäng av vårt dataset för att göra det enklare att träna modellen.

In [ ]:
train = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv')
validation = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')

train.drop(['severe_toxic','obscene','threat','insult','identity_hate'], axis=1, inplace=True) # droppar klassificeringar av hatfulla kommentarer

In [ ]:
train = train.loc[:15000,:] #bestämmer delmäng
train.shape #hämtar form på tränings set

Vi undersöker vad max antal tecken kan vara i en kommentar vilket kan underlätta för oss senare med padding.

In [ ]:
train['comment_text'].apply(lambda x:len(str(x).split())).max() # hämtar max antal tecken i kommentar

Förbereder vår data

In [ ]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.comment_text.values, train.toxic.values, 
                                                  stratify=train.toxic.values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

Från vad jag har sett här på Kaggle så används AUC score för själva valideringen, nedanstående kod skapar en funktion som hämtar AUC score som kommer att användas. AUC värdet ligger mellan 0 och 1, en modell med 100% fel predictions har ett AUC på 0.0; en modell som har 100% rätt har en AUC på 1.0. AUC är därmed ett sätt att mäta prestation för en modell och är ett sammanlagt mått på prestanda över alla möjliga klassificeringsgränser.

*"AUC stands for "Area under the ROC Curve." That is, AUC measures the entire two-dimensional area underneath the entire ROC curve (think integral calculus) from (0,0) to (1,1)." - Machine Learning Crash Course Google*

In [ ]:
def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''
    
    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

Tokenization

I en RNN så sker vår input av en mening ord för ord, keras tokenizer tar alla unika ord och formar i princip en ordbok med ord och dess antal förekomster som värden. Sedan så sorteras ordboken i fallande ordningsföljd och tilldelar det första värdet 1, andra värdet 2 och så vidare. Till exempel, vi säger att av alla kommentarer så användes ordet "article" mest, den hade då fått index 1 och vektorn som representerar order "article" hade varit en så kallad "one-hot" vektor med värdet 1 på position 1, resterande värden 0.

In [ ]:
# använder keras tokenizer
token = text.Tokenizer(num_words=None)
max_len = 1500

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

#zero paddar sekvenserna
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

Nedanstående kod försöker att använda Kaggle TPU och därefter returnerna lämplig distruberings strategi

In [ ]:
# Upptäck hårdvara, returnera lämplig distributionsstrategi
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Prövar först med en enkel modell för att se vilket resultat som nås

model.Sequential() säger åt keras att vi kommer bygga vårt nätverk sekventiellt. Sedan så lägger vi först till vårt Embedding lager som är ett lager av neuroner som tar in en input som är en 'one hot' vektor av varje ord och konverterar den till en 300 dimensionell vektor. Den ger oss word embedding som liknar word2vec - word2vec gör om text till en numerisk form så att neural networks kan förstå. I denna modell så är vårt embedding lager inte pre-trained; pre-trained embedding är en embedding som har tränats i en task för att sedan användas i en liknande task. Dessa embeddings är tränade på stora datauppsättningar, lagrade, och används sedan för att lösa andra problem. Sedan så har vi ett SimpleRNN lager med 100 units, notera att vi i denna modell inte använder dropout. Sist så har vi vårt Dense layer med en sigmoid aktiveringsfunktion som tar outputen från det tidigare lagret för att göra en prediction.

In [ ]:
# En enkel modell (SimpleRNN) med ett dense layer

with strategy.scope():
    
    model = Sequential() #sekventiellt nätverk
    model.add(Embedding(len(word_index) + 1, #konverterar till 300 dimensionell vektor
                     300,
                     input_length=max_len))
    model.add(SimpleRNN(100))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.summary()

Träning av modellen

In [ ]:
history = model.fit(xtrain_pad, ytrain, nb_epoch=5, batch_size=64)

Beräkna AUC Score

In [ ]:
scores = model.predict(xvalid_pad)
print("AUC: %.2f%%" % (roc_auc(scores,yvalid)))

Vi använder oss utav Word embeddings

Word embeddings är en typ av ord representation som tillåter ord med liknande mening att ha liknande representation

Mer specifikt så använder vi oss utav GloVe, vilket är ett av de förtränade embeddings som jag tidigare nämnde i vår mer enkla modell. För att förenkla allt så kan man säga att meningen bakom GloVe som ett pre-trained word embedding är att härleda förhållandet mellan ord från global statistik.

"GloVe is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus, and the resulting representations showcase interesting linear substructures of the word vector space." - Stanford

In [ ]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('/kaggle/input/glove840b300dtxt/glove.840B.300d.txt','r',encoding='utf-8') #precis som vår data för kommentarer så lägger vi till GloVe i vår Kaggle-databas
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
# skapa en embedding matris för orden vi har i vår datauppsättning
embedding_matrix = np.zeros((len(word_index) + 1,300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Mer komplex modell

Först så beräknar vi vår embedding matris för vår orduppsättning från den pre-trained GloVe vektorn. Sedan, medans vi bygger vårt embedding lager så passerar vi embedding matrisen som weights till lagret istället för att träna den över orduppsättningen; därför så sätter vi trainable = False. Resterande delar av modellen är likande bortsett från att vi nu anävnder 3 stycken LSTM layers med 100 units istället för en enstaka SimpleRNN. Jag har valt i båda modellerna att använda mig av binary crossentropy som förlustfunktion, detta är för att det vi försöker lösa är, om vi förenklar det, binärt. Med det menar jag att antingen så är en kommentar toxic (1) eller non-toxic (0). Förlustfunktionen binary crossentropy kändes då lämplig då den används för ja/nej klassificeringar.

In [ ]:
%%time
with strategy.scope():
    
    model = Sequential() # sekventiellt nätverk
    model.add(Embedding(len(word_index) + 1, # embedding lager som nu använder GloVe istället
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))

    model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3,return_sequences=True)) # tre LSTM lager med 100 units, dropout har lagts till
    model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3,return_sequences=True))
    model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.summary()

In [ ]:
model.fit(xtrain_pad, ytrain, nb_epoch=5, batch_size=64)

In [ ]:
scores = model.predict(xvalid_pad)
print("Auc: %.2f%%" % (roc_auc(scores,yvalid)))

# Resultat

Results – What answer was found to the research question; what did the study find? Was the tested hypothesis true?

**Resultat första, enkla, modell**

Epoch 5/5

12000/12000 [==============================] - 189s 16ms/step - loss: 0.0015 - accuracy: 0.9999

AUC: 0.89

Vi kan se att vår modell når en noggrannhet på 1 vilket självklart tyder på att vi övertränar extremt mycket. Detta visar att vi bör kanske använda en mer avancerad modell och t.ex. använda dropout för att nå bättre resultat. Även fast modellen är simpel och övertränar så nåddes fortfarande ett AUC score på 0.82 utan mycket ansträngning.

**Resultat andra, mer komplexa, modell**

Epoch 5/5
12000/12000 [==============================] - 801s 67ms/step - loss: 0.1069 - accuracy: 0.9589

AUC: 0.97

Denna modell presterar bättre jämfört med den tidigare. Tittar vi på vår förlust och noggrannhet för båda modellerna så ser vi en stor skillnad i resultat.

Ursprunglig modell:

loss: 0.0015 accuracy: 0.9999

Ny modell:

loss: 0.1069 accuracy: 0.9589

Resultaten från den första modellen är icke-trovärdigt och visar tecken på överträning, om ens noggrannhet ligger på 1 så är det något som inte stämmer helt enkelt.

I vår andra modell så ser resultaten mycket mer trovärdiga ut med en förlust på 0.1069 och en noggrannhet på 0.9589. Samt så nådde vi en AUC på 0.97 vilket är betydligt bättre än vår första modell som till och med var extremt övertränad. Detta syftar på att vår nya modell har presterat bra för vårt problem.


# Diskussion

**Vad för slutsatser kan vi göra?**

Det är klart och tydligt att en allt för simpel modell inte kommer prestera bra för vårt problem då det är en stor datamängd och ett komplext problem. Vår andra modell som var lite mer anpassad för problemet presterade bättre och nådde bra resultet för AUC. Dock så är såklart inte modellen perfekt på något vis, jag tror att det går att lösa problemet allt bättre med en mer advancerad lösning/modell. Något som kommer till tanke efter duggorna vi hade är BERT och dess förmåga att lösa sådana här problem effektivt. BERT har setts som en ny era inom Natural Language Processing (NLP) och är en modell som har slagit nya rekord för hur bra en modell kan hantera språkbaserade problem. BERT-modellen är open-sourced och därmed tillgänglig för alla att ladda ner versioner av modellen som redan har blivit tränade på stora datauppsättningar. Detta gör det möjligt för att vem som helst som bygger en machine learning modell för språkbehandling att använda denna kraftfulla komponent vilket sparar tid, energi, kunskap och resurser som hade gått till att träna en språkbehandlings modell från grunden. Jag skulle säga att mitt perspektiv framåt är att implementera BERT för detta problem och se vad man hade fått för resultat och hur smidigt det skulle vara jämfört med mitt ursprungliga tillvägagångsätt. Jag har inte jobbat med NLP till denna grad förr men det verkar väldigt intressant och användbart, detta känns som att detta är endast ett scenario där det kan vara smart att implementera lösningar med NLP. Hatfulla kommentarar tror jag påverkar människor allt mer än vad man tror och ser, speciellt online där människor oftast är anonyma när dessa kommentarer görs. Det går från person till person, personligen tror jag inte att jag skulle bli allt för påverkad av sådana kommentarer, men det är naturligt att många människor skulle bli det och därför så är det viktigt att sådana här problem undersöks mer så att vi effektivt kan motverka hatfulla kommentarer online för att människor ska få en bättre upplevelse!


**Litteratur och kod som använts:**

Binary crossentropy

https://peltarion.com/knowledge-center/documentation/modeling-view/build-an-ai-model/loss-functions/binary-crossentropy

Pretrained word embedding NLP

https://www.analyticsvidhya.com/blog/2020/03/pretrained-word-embeddings-nlp/

Word2Vec

https://pathmind.com/wiki/word2vec

GloVe (datauppsättning och information)

https://nlp.stanford.edu/projects/glove/

AUC

https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc

NLP och Wordcloud's

https://www.kaggle.com/arthurtok/spooky-nlp-and-topic-modelling-tutorial

Kaggle Tävlingen som detta projekt är baserat på, datauppsättningen har hämtats härifrån

https://www.kaggle.com/c/jigsaw-multilingual-toxic-comment-classification/overview

Exploratory Data Analysis (EDA), bra förklaring och visualisering av data.

https://www.kaggle.com/jagangupta/stop-the-s-toxic-comments-eda

En kernel för liknande äldre tävling

https://www.kaggle.com/rhodiumbeng/classifying-multi-label-comments-0-9741-lb
